In [5]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

# TEST_DATA_PATH = os.path.join(DATA_PATH, "stowing-result-bak-mmmp-singleproc")
TEST_DATA_PATH = os.path.join(DATA_PATH, "stowing-result")
try_mkdir(TEST_DATA_PATH)

HEADS = ["plan_time", "length", "MP_tot", "MP_count", "failed_MPs", "IK_tot", "IK_count", "failed_IKs", "success"]
DISPS = ["Time", "Actions", "MP_tot", "MPs", "MP fails", "IK_tot", "IKs", "IK fails", "Failures", "Total"]
H2D = {}
D2H = {}
for head, disp in zip(HEADS, DISPS):
    H2D[head] = disp
    D2H[disp] = head

RTYPE = "indy7gripper"
DAT_DIR = '20210709-172009'
# DAT_DIR = '20210707-034434'


# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
CNAME_EXCLUDE = 'None'
GRASP_SAMPLE = 100
VISUALIZE = True
PLAY_RESULT = False
SHOW_STATE = False

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100

ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

res_dict_all = defaultdict(lambda: defaultdict(list))
resfile_list = sorted(os.listdir(os.path.join(DATASET_PATH)))
for resfile in resfile_list:
    args = resfile[:-4].split("_")[1:]
    f_option = "_".join(args[:-2])
    fid, cname = args[-2:]
    resdat = load_pickle(os.path.join(DATASET_PATH, resfile))
    res_dict_all[f_option][cname].append([resdat[head] if head in resdat else None for head in HEADS])
    
exp_list = ['plane_3', 'plane_5', 'obj_3', 'obj_3_pole']
for ekey in exp_list:
    if ekey not in res_dict_all:
        continue
    res_dict = res_dict_all[ekey]
    
    succ_vec_list = []
    len_list = []
    None_cols = []
    CNAME_LIST_CUR = [cname for cname in CNAME_LIST if cname in res_dict]
    for cname in CNAME_LIST_CUR:
        vv = np.array(res_dict[cname])
        if len(vv)>0:
            None_cols.append(np.any(np.equal(vv, None), axis=0))
            succ_vec = vv[:,-1]
            succ_vec_list.append(succ_vec)
            len_list.append(len(succ_vec))
            if cname == CNAME_EXCLUDE:
                fail_vec = np.logical_not(succ_vec)
                idc_fail_none = np.where(fail_vec)[0]
    print("="*30 + " {:<10} ".format(ekey) + "="*30)
    if len(len_list)>1:
        None_cols = np.any(None_cols, 0)
        good_cols = np.where(np.logical_not(None_cols))[0]
        idc_include = [i_cn for i_cn, cname in enumerate(CNAME_LIST_CUR) if cname != CNAME_EXCLUDE]
        min_len = np.min(np.array(len_list)[idc_include])
        succ_vec_all = np.all([svec[:min_len] for svec in 
                               np.array(succ_vec_list)[idc_include]], axis=0)
        idc_succ_all = np.where(succ_vec_all)[0]

        print(("{:<10}:"+ " {:<10}"*len(good_cols)).format(
            "Checker", *[DISPS[gidx] for gidx in good_cols])+" {:<10}".format(DISPS[-1])
              + "(valid={})".format(len(idc_succ_all))
             )
        hard_vec_list = []
        for cname in CNAME_LIST_CUR:
            vv = np.array(res_dict[cname])[:, good_cols]
            if len(vv)>0:
                succ_vec = vv[:,-1]
                fail_vec = np.logical_not(succ_vec)
                hard_vec_list.append(vv[:min_len, 0] > np.mean(vv[:min_len, 0])*1.0)
                if cname == CNAME_EXCLUDE:
                    idc_succ_now = np.where(succ_vec)[0]
                    idc_fail_none = np.where(fail_vec)[0]
                else:
                    idc_succ_now = idc_succ_all
                print(
                    "{:<10}:".format(cname) 
                    + (" {:<10}"*len(good_cols)+" {:<10}").format(
                        *(list(np.round(np.mean(vv[idc_succ_now, :-1], axis=0).astype(np.float), 2)) \
                          +[np.sum(fail_vec), len(vv)])))

#         print("="*30 + " {:<10} ".format(ekey+"-all") + "="*30)
#         print(("{:<10}:"+ " {:<10}"*len(good_cols)).format(
#             "Checker", *[DISPS[gidx] for gidx in good_cols])+" {:<10}".format(DISPS[-1]))
#         hard_vec_all = np.sum(hard_vec_list, axis=0)>=1
#         idc_hard_all = np.where(hard_vec_all)[0]
#         idc_hard = np.where(np.logical_and(succ_vec_all, hard_vec_all))[0]
#         for cname in CNAME_LIST:
#             if cname in res_dict:
#                 vv = np.array(res_dict[cname])[:, good_cols]
#                 if len(vv)>0:
#                     succ_vec = vv[:,-1].astype(np.bool)
#                     fail_vec = np.logical_not(succ_vec).astype(np.bool)
#                     idc_succ_now = np.where(succ_vec)[0]
#                     print(
#                         "{:<10}:".format(cname) 
#                         + (" {:<10}"*len(good_cols)+" {:<10}").format(
#                             *(list(np.round(np.mean(vv[idc_succ_now, :-1].astype(np.float), axis=0).astype(np.float), 2)) \
#                               +[np.sum(fail_vec), len(succ_vec)])))

============================== plane_3    ==============================
Checker   : Time       Actions    MPs        MP fails   Failures   Total     (valid=50)
None      : 58.47      2.64       20.19      11.4       3          50        
Tool      : 40.53      3.44       18.34      7.82       0          50        
ToolReach : 7.21       2.84       11.76      1.1        0          50        
Full      : 6.9        3.16       11.56      1.06       0          50        
============================== plane_5    ==============================
Checker   : Time       Actions    MPs        MP fails   Failures   Total     (valid=41)
Tool      : 36.61      2.63       18.0       6.98       8          50        
ToolReach : 20.34      3.8        35.37      3.27       1          50        
Full      : 11.55      3.22       22.8       1.61       2          50        


# MMMP 300s single proc indy
```
============================== plane_3    ==============================
Checker   : Time       Actions    MPs        MP fails   Failures   Total     (valid=50)
None      : 58.47      2.64       20.19      11.4       3          50        
Tool      : 40.53      3.44       18.34      7.82       0          50        
ToolReach : 7.21       2.84       11.76      1.1        0          50        
Full      : 6.9        3.16       11.56      1.06       0          50        
============================== plane_5    ==============================
Checker   : Time       Actions    MPs        MP fails   Failures   Total     (valid=41)
Tool      : 36.61      2.63       18.0       6.98       8          50        
ToolReach : 20.34      3.8        35.37      3.27       1          50        
Full      : 11.55      3.22       22.8       1.61       2          50    
============================== obj_3      ==============================
Checker   : Time       Actions    MPs        MP fails   Failures   Total     (valid=44)
None      : 128.26     3.07       34.18      25.14      22         50        
Tool      : 88.22      4.32       42.11      16.61      6          50        
ToolReach : 32.21      4.95       42.02      5.07       0          50        
Full      : 30.39      5.05       45.34      4.34       0          50        
============================== obj_3_pole ==============================
Checker   : Time       Actions    MPs        MP fails   Failures   Total     (valid=29)
Tool      : 111.91     4.21       46.93      20.97      20         50        
ToolReach : 47.87      4.83       48.14      7.66       3          50        
Full      : 40.91      5.24       53.07      5.59       3          50      
```